In [84]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from wordcloud import WordCloud
import warnings
from PIL import Image
import re
import copy
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
warnings.filterwarnings('ignore')
dataset = pd.read_excel('nurdata.xlsx')


In [94]:
dataset.head()

,eventid,iyear,imonth,iday,country,country_txt,region,crit1,crit2,crit3,...,nkill,nkillus,property,propextent,propextent_txt,nreleased,INT_LOG,INT_IDEO,INT_MISC,INT_ANY
0,201401010001,2014,1,1,45,Colombia,3,1,1,1,...,0.0,0.0,1,4.0,Unknown,NaN,0,0,0,0
1,201401010002,2014,1,1,182,Somalia,11,1,1,1,...,4.0,0.0,1,3.0,Minor (likely < $1 million),NaN,0,0,0,0
2,201401010003,2014,1,1,153,Pakistan,6,1,1,1,...,3.0,0.0,1,3.0,Minor (likely < $1 million),NaN,0,0,0,0
3,201401010004,2014,1,1,153,Pakistan,6,1,1,1,...,0.0,0.0,0,NaN,NaN,NaN,-9,-9,0,-9
4,201401010005,2014,1,1,182,Somalia,11,1,1,1,...,NaN,0.0,0,NaN,NaN,3.0,0,0,0,0


In [95]:
# max_size = dataset['attacktype1'].max()
# def ohe(index):
#     encoding = [0]*max_size
#     encoding[index-1] = 1
#     return encoding
# dataset['attacktype1_ohe'] = dataset['attacktype1'].map(ohe)
# dataset

In [96]:
dataset = pd.get_dummies(dataset, columns=['attacktype1'])
dataset = pd.get_dummies(dataset, columns=['targtype1'])
dataset = pd.get_dummies(dataset, columns=['weaptype1'])

In [97]:
dataset.columns 

Index(['eventid', 'iyear', 'imonth', 'iday', 'country', 'country_txt',
       'region', 'crit1', 'crit2', 'crit3', 'doubtterr', 'alternative',
       'alternative_txt', 'success', 'attacktype1_txt', 'targtype1_txt',
       'gname', 'weaptype1_txt', 'nkill', 'nkillus', 'property', 'propextent',
       'propextent_txt', 'nreleased', 'INT_LOG', 'INT_IDEO', 'INT_MISC',
       'INT_ANY', 'attacktype1_1', 'attacktype1_2', 'attacktype1_3',
       'attacktype1_4', 'attacktype1_5', 'attacktype1_6', 'attacktype1_7',
       'attacktype1_8', 'attacktype1_9', 'targtype1_1', 'targtype1_2',
       'targtype1_3', 'targtype1_4', 'targtype1_5', 'targtype1_6',
       'targtype1_7', 'targtype1_8', 'targtype1_9', 'targtype1_10',
       'targtype1_11', 'targtype1_12', 'targtype1_13', 'targtype1_14',
       'targtype1_15', 'targtype1_16', 'targtype1_17', 'targtype1_18',
       'targtype1_19', 'targtype1_20', 'targtype1_21', 'targtype1_22',
       'weaptype1_2', 'weaptype1_5', 'weaptype1_6', 'weaptype1_7',
  

In [98]:
network_data = dataset[['eventid', 'iyear', 'imonth', 'iday', 'country', 'country_txt',
       'region', 'crit1', 'crit2', 'crit3', 'doubtterr', 'alternative',
       'alternative_txt', 'success', 'attacktype1_txt', 'targtype1_txt',
       'gname', 'weaptype1_txt', 'nkill', 'nkillus', 'property', 'propextent',
       'propextent_txt', 'nreleased', 'INT_LOG', 'INT_IDEO', 'INT_MISC',
       'INT_ANY', 'attacktype1_1', 'attacktype1_2', 'attacktype1_3',
       'attacktype1_4', 'attacktype1_5', 'attacktype1_6', 'attacktype1_7',
       'attacktype1_8', 'attacktype1_9', 'targtype1_1', 'targtype1_2',
       'targtype1_3', 'targtype1_4', 'targtype1_5', 'targtype1_6',
       'targtype1_7', 'targtype1_8', 'targtype1_9', 'targtype1_10',
       'targtype1_11', 'targtype1_12', 'targtype1_13', 'targtype1_14',
       'targtype1_15', 'targtype1_16', 'targtype1_17', 'targtype1_18',
       'targtype1_19', 'targtype1_20', 'targtype1_21', 'targtype1_22',
       'weaptype1_2', 'weaptype1_5', 'weaptype1_6', 'weaptype1_7',
       'weaptype1_8', 'weaptype1_9', 'weaptype1_10', 'weaptype1_11',
       'weaptype1_12', 'weaptype1_13']]


In [99]:
dataset

,eventid,iyear,imonth,iday,country,country_txt,region,crit1,crit2,crit3,...,weaptype1_2,weaptype1_5,weaptype1_6,weaptype1_7,weaptype1_8,weaptype1_9,weaptype1_10,weaptype1_11,weaptype1_12,weaptype1_13
0,201401010001,2014,1,1,45,Colombia,3,1,1,1,...,0,0,1,0,0,0,0,0,0,0
1,201401010002,2014,1,1,182,Somalia,11,1,1,1,...,0,0,1,0,0,0,0,0,0,0
2,201401010003,2014,1,1,153,Pakistan,6,1,1,1,...,0,0,1,0,0,0,0,0,0,0
3,201401010004,2014,1,1,153,Pakistan,6,1,1,1,...,0,0,1,0,0,0,0,0,0,0
4,201401010005,2014,1,1,182,Somalia,11,1,1,1,...,0,0,0,0,0,0,0,0,0,1
5,201401010007,2014,1,1,95,Iraq,10,1,1,0,...,0,0,1,0,0,0,0,0,0,0
6,201401010017,2014,1,1,95,Iraq,10,1,1,0,...,0,1,0,0,0,0,0,0,0,0
7,201401010018,2014,1,1,95,Iraq,10,1,1,1,...,0,1,0,0,0,0,0,0,0,0
8,201401010020,2014,1,1,92,India,6,1,1,1,...,0,0,1,0,0,0,0,0,0,0
9,201401010021,2014,1,1,92,India,6,1,1,1,...,0,0,1,0,0,0,0,0,0,0


In [100]:
# X = dataset.iloc[:, :-1].values
# le = LabelEncoder()
# X[:, 15] = le.fit_transform(X[:, 15])
# ohe = OneHotEncoder(categorical_features = [15])
# X = ohe.fit_transform(X).toarray()


In [101]:
abb = list(dataset.groupby('gname').count()['eventid'].sort_values(ascending=False)[51:].index)
abb
dataset['gname'].value_counts()

Unknown                                                        25528
Islamic State of Iraq and the Levant (ISIL)                     5239
Taliban                                                         4243
Al-Shabaab                                                      2402
Boko Haram                                                      1608
New People's Army (NPA)                                         1141
Houthi extremists (Ansar Allah)                                 1027
Maoists                                                          999
Kurdistan Workers' Party (PKK)                                   920
Donetsk People's Republic                                        624
Al-Qaida in the Arabian Peninsula (AQAP)                         548
Fulani extremists                                                491
Tehrik-i-Taliban Pakistan (TTP)                                  469
Sinai Province of the Islamic State                              445
Muslim extremists                 

In [102]:
# dataset['gname'].filter(like= 'Taliban')
# datasett = dataset[dataset.gname != 'Taliban']
datasett = dataset
for a in abb:
    place = a
    datasett = datasett[datasett.gname != place]
    print(place)

# dataset = dataset.drop("Tutsi extremists")

# dataset["gname"].value_counts() 
# for a in dataset["gname"]:
#     if a == "Taliban":
#         del a
# dataset["index"].value_counts() 


Dissident Republicans
Ansar al-Sharia (Libya)
Jamaat Nusrat al-Islam wal Muslimin (JNIM)
Lashkar-e-Jhangvi
National Democratic Front of Bodoland (NDFB)
Democratic Front for the Liberation of Rwanda (FDLR)
Asa'ib Ahl al-Haqq
United Baloch Army (UBA)
Ansar Bayt al-Maqdis (Ansar Jerusalem)
Al-Qaida in the Islamic Maghreb (AQIM)
Paraguayan People's Army (EPP)
Niger Delta Avengers (NDA)
Ansar al-Dine (Mali)
Patriotic Resistance Front in Ituri (FRPI)
Mayi Mayi
Adan-Abyan Province of the Islamic State
Arakan Rohingya Salvation Army (ARSA)
National Socialist Council of Nagaland-Isak-Muivah (NSCN-IM)
People's Liberation Front of India
Lashkar-e-Islam (Pakistan)
National Socialist Council of Nagaland-Khaplang (NSCN-K)
Hay'at Tahrir al-Sham
People's Liberation Army (India)
Sudan People's Liberation Movement - North
Islamic State in Bangladesh
Islamic Front (Syria)
Al-Naqshabandiya Army
Jamaat-E-Islami (Bangladesh)
Muslim Brotherhood
Seleka
The New Irish Republican Army
Ajnad Misr
Maute Group
Jays

Kazakhstan Liberation Army
National Socialist Council of Nagaland-Khole-Kitovi (NSCN-KK)
Right-Wing Paramilitaries
Black Hebrew Israelites
Incel extremists
Mai Mai Charles Militia
United Tribal Liberation Army (UTLA)
Freital Group
Makhachkala Gang
Army of the Tribes
Ansar Al-Khilafa (Philippines)
Mapuche Ancestral Resistance (RAM)
Mozambique Liberation Front (FRELIMO)
Movement of Democratic Forces of Casamance
Concerned Militant Leaders (CML)
Harkatul Jihad-e-Islami
Chechen Rebels
Anas al-Dabbashi Brigade
Christian extremists
Hadramawt Tribes Alliance
Moro National Liberation Front (MNLF)
United Front for Democracy Against Dictatorship
Aceh Singkil Islamic Care Youth Students Association (PPI)
Mustafa al-Hujairi Group
Mai Mai Yakutumba Militia
Revolutionary Cells Network (SRN)
Ahle Sunnat Wal Jamaat (ASWJ-Pakistan)
Revolution's Brigade
Bunda Dia Kongo (BDK)
Hynniewtrep National Liberation Council (HNLC)
Free Libya Martyrs Brigade
All Nepal National Free Student Union-Revolutionary
Revo

Tehama Movement
Tehrik-e-Hurriyat (TeH)
Tehrik-e-Taliban Islami (TTI)
Thadou People's Liberation Army (TPLA)
Thanthai Periyar Dravidar Kazhagam (TPDK)
The 78 Unemployed
The Independent Military Wing of the Syrian Revolution Abroad
The Irish Volunteers
The Justice Department
The Nocturnals
The Owners of the White Flags
The Third Way (Der III. Weg)
Tigray Peoples Liberation Front (TPLF)
Tiv Militia
New Indigenous People's Army
Nazhdak
Earth Liberation Front (ELF)
Jai Shri Ram Hindu Bhai Group
Hutu extremists
Indian Mujahideen
Indigenous People's Front of Tripura (IPFT)
Individuals Tending Toward Savagery
Informal Feminist Commando for Anti-authoritarian Action
Ingush Rebels
International Revolutionary Front
Irrintzi
Islamic Movement (Nigeria)
Islamic Movement of Uzbekistan (IMU)
Jaish Tahkim al-Din
Hizb al-Tahrir al-Islami (HT)
Jaish Usama
Jaish al-Islam (Libya)
Jaish al-Umar (JaU)
Jaish as-Saiyouf (Army of Swords)
Jammu and Kashmir Islamic Front
Janatantrik Tiger Janashakti Party
Jemaah

In [103]:
datasett = datasett[datasett.gname != "Unknown"]

datasett['gname'].value_counts()
# for b in abb:
#     dataset.filter(like= b, axis = "gname")
    
# # dataset["gname"].value_counts()
# for name in dataset["gname"]:
#     for b in abb:
#         if name == b:
#             print (name)
#         else: 
#              del name

#     del dataset["gname"]




Islamic State of Iraq and the Levant (ISIL)                   5239
Taliban                                                       4243
Al-Shabaab                                                    2402
Boko Haram                                                    1608
New People's Army (NPA)                                       1141
Houthi extremists (Ansar Allah)                               1027
Maoists                                                        999
Kurdistan Workers' Party (PKK)                                 920
Donetsk People's Republic                                      624
Al-Qaida in the Arabian Peninsula (AQAP)                       548
Fulani extremists                                              491
Tehrik-i-Taliban Pakistan (TTP)                                469
Sinai Province of the Islamic State                            445
Muslim extremists                                              411
Palestinian Extremists                                        

In [104]:
# max_size = 10
# def ohe(index):
#     encoding = [0]*max_size
#     encoding[index-1] = 1
#     return encoding
# datasett['gname_ohe'] = datasett['gname'].map(ohe)
# datasett
datasett = pd.get_dummies(datasett, columns=['country'])
for col in datasett.columns:
    print("'"+col+"'"+",")

'eventid',
'iyear',
'imonth',
'iday',
'country_txt',
'region',
'crit1',
'crit2',
'crit3',
'doubtterr',
'alternative',
'alternative_txt',
'success',
'attacktype1_txt',
'targtype1_txt',
'gname',
'weaptype1_txt',
'nkill',
'nkillus',
'property',
'propextent',
'propextent_txt',
'nreleased',
'INT_LOG',
'INT_IDEO',
'INT_MISC',
'INT_ANY',
'attacktype1_1',
'attacktype1_2',
'attacktype1_3',
'attacktype1_4',
'attacktype1_5',
'attacktype1_6',
'attacktype1_7',
'attacktype1_8',
'attacktype1_9',
'targtype1_1',
'targtype1_2',
'targtype1_3',
'targtype1_4',
'targtype1_5',
'targtype1_6',
'targtype1_7',
'targtype1_8',
'targtype1_9',
'targtype1_10',
'targtype1_11',
'targtype1_12',
'targtype1_13',
'targtype1_14',
'targtype1_15',
'targtype1_16',
'targtype1_17',
'targtype1_18',
'targtype1_19',
'targtype1_20',
'targtype1_21',
'targtype1_22',
'weaptype1_2',
'weaptype1_5',
'weaptype1_6',
'weaptype1_7',
'weaptype1_8',
'weaptype1_9',
'weaptype1_10',
'weaptype1_11',
'weaptype1_12',
'weaptype1_13',
'country_4',
'cou

In [105]:
datasett = pd.get_dummies(datasett, columns=['gname'])

In [118]:
for col in datasett.columns:
    print("'"+col+"'"+",")

'eventid',
'iyear',
'imonth',
'iday',
'country_txt',
'region',
'crit1',
'crit2',
'crit3',
'doubtterr',
'alternative',
'alternative_txt',
'success',
'attacktype1_txt',
'targtype1_txt',
'weaptype1_txt',
'nkill',
'nkillus',
'property',
'propextent',
'propextent_txt',
'nreleased',
'INT_LOG',
'INT_IDEO',
'INT_MISC',
'INT_ANY',
'attacktype1_1',
'attacktype1_2',
'attacktype1_3',
'attacktype1_4',
'attacktype1_5',
'attacktype1_6',
'attacktype1_7',
'attacktype1_8',
'attacktype1_9',
'targtype1_1',
'targtype1_2',
'targtype1_3',
'targtype1_4',
'targtype1_5',
'targtype1_6',
'targtype1_7',
'targtype1_8',
'targtype1_9',
'targtype1_10',
'targtype1_11',
'targtype1_12',
'targtype1_13',
'targtype1_14',
'targtype1_15',
'targtype1_16',
'targtype1_17',
'targtype1_18',
'targtype1_19',
'targtype1_20',
'targtype1_21',
'targtype1_22',
'weaptype1_2',
'weaptype1_5',
'weaptype1_6',
'weaptype1_7',
'weaptype1_8',
'weaptype1_9',
'weaptype1_10',
'weaptype1_11',
'weaptype1_12',
'weaptype1_13',
'country_4',
'country_5',


In [137]:
datasettt = datasett[['success','attacktype1_1',
       'attacktype1_2', 'attacktype1_3', 'attacktype1_4', 'attacktype1_5',
       'attacktype1_6', 'attacktype1_7', 'attacktype1_8', 'attacktype1_9',
       'targtype1_1', 'targtype1_2', 'targtype1_3', 'targtype1_4',
       'targtype1_5', 'targtype1_6', 'targtype1_7', 'targtype1_8',
       'targtype1_9', 'targtype1_10', 'targtype1_11', 'targtype1_12',
       'targtype1_13', 'targtype1_14', 'targtype1_15', 'targtype1_16',
       'targtype1_17', 'targtype1_18', 'targtype1_19', 'targtype1_20',
       'targtype1_21', 'targtype1_22', 'weaptype1_2', 'weaptype1_5',
       'weaptype1_6', 'weaptype1_7', 'weaptype1_8', 'weaptype1_9',
       'weaptype1_10', 'weaptype1_11', 'weaptype1_12', 'weaptype1_13', 'nkill', 'country_4',
'country_5',
'country_6',
'country_14',
'country_15',
'country_16',
'country_18',
'country_19',
'country_21',
'country_28',
'country_32',
'country_33',
'country_37',
'country_38',
'country_41',
'country_42',
'country_43',
'country_45',
'country_55',
'country_56',
'country_60',
'country_65',
'country_68',
'country_69',
'country_74',
'country_75',
'country_78',
'country_92',
'country_93',
'country_94',
'country_95',
'country_97',
'country_98',
'country_102',
'country_104',
'country_106',
'country_107',
'country_110',
'country_113',
'country_121',
'country_122',
'country_123',
'country_130',
'country_137',
'country_138',
'country_141',
'country_142',
'country_146',
'country_147',
'country_151',
'country_153',
'country_155',
'country_156',
'country_160',
'country_161',
'country_167',
'country_173',
'country_182',
'country_183',
'country_185',
'country_186',
'country_195',
'country_198',
'country_200',
'country_201',
'country_202',
'country_203',
'country_205',
'country_208',
'country_209',
'country_210',
'country_213',
'country_214',
'country_217',
'country_228',
'country_229',
'country_603',
'country_1004','crit1', 'crit2', 'crit3', 'gname_Abu Sayyaf Group (ASG)',
'gname_Al-Nusrah Front',
'gname_Al-Qaida in the Arabian Peninsula (AQAP)',
'gname_Al-Shabaab',
'gname_Allied Democratic Forces (ADF)',
'gname_Anarchists',
'gname_Anti-Balaka Militia',
'gname_Anti-Muslim extremists',
'gname_Baloch Liberation Army (BLA)',
'gname_Baloch Liberation Front (BLF)',
'gname_Baloch Republican Army (BRA)',
'gname_Bangsamoro Islamic Freedom Movement (BIFM)',
'gname_Barisan Revolusi Nasional (BRN)',
'gname_Barqa Province of the Islamic State',
'gname_Boko Haram',
'gname_Communist Party of India - Maoist (CPI-Maoist)',
'gname_Communist Party of Nepal - Maoist (CPN-Maoist-Chand)',
"gname_Donetsk People's Republic",
'gname_Free Syrian Army',
'gname_Fulani extremists',
'gname_Garo National Liberation Army',
'gname_Gorkha Janmukti Morcha (GJM)',
'gname_Hamas (Islamic Resistance Movement)',
'gname_Hizbul Mujahideen (HM)',
'gname_Houthi extremists (Ansar Allah)',
'gname_Islamic State of Iraq and the Levant (ISIL)',
'gname_Janjaweed',
'gname_Jihadi-inspired extremists',
'gname_Khorasan Chapter of the Islamic State',
"gname_Kurdistan Workers' Party (PKK)",
'gname_Lashkar-e-Taiba (LeT)',
"gname_Lord's Resistance Army (LRA)",
"gname_Luhansk People's Republic",
'gname_Maoists',
'gname_Mozambique National Resistance Movement (MNR)',
'gname_Muslim extremists',
'gname_National Liberation Army of Colombia (ELN)',
"gname_New People's Army (NPA)",
'gname_Palestinian Extremists',
'gname_Pro Hartal Activists',
'gname_Revolutionary Armed Forces of Colombia (FARC)',
'gname_Runda Kumpulan Kecil (RKK)',
'gname_Separatists',
'gname_Sinai Province of the Islamic State',
"gname_Sudan People's Liberation Movement in Opposition (SPLM-IO)",
'gname_Taliban',
'gname_Tehrik-i-Taliban Pakistan (TTP)',
'gname_Tribesmen',
'gname_Tripoli Province of the Islamic State',
'gname_United Liberation Front of Assam (ULFA)']]
datasettt = datasettt.dropna(how= 'any')

In [138]:
datasettt.head()

,success,attacktype1_1,attacktype1_2,attacktype1_3,attacktype1_4,attacktype1_5,attacktype1_6,attacktype1_7,attacktype1_8,attacktype1_9,...,gname_Revolutionary Armed Forces of Colombia (FARC),gname_Runda Kumpulan Kecil (RKK),gname_Separatists,gname_Sinai Province of the Islamic State,gname_Sudan People's Liberation Movement in Opposition (SPLM-IO),gname_Taliban,gname_Tehrik-i-Taliban Pakistan (TTP),gname_Tribesmen,gname_Tripoli Province of the Islamic State,gname_United Liberation Front of Assam (ULFA)
0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [157]:
X = datasettt[['attacktype1_1',
       'attacktype1_2', 'attacktype1_3', 'attacktype1_4', 'attacktype1_5',
       'attacktype1_6', 'attacktype1_7', 'attacktype1_8', 'attacktype1_9', 'weaptype1_2', 'weaptype1_5',
       'weaptype1_6', 'weaptype1_7', 'weaptype1_8', 'weaptype1_9',
       'weaptype1_10', 'weaptype1_11', 'weaptype1_12', 'weaptype1_13',  'targtype1_1', 'targtype1_2', 'targtype1_3', 'targtype1_4',
       'targtype1_5', 'targtype1_6', 'targtype1_7', 'targtype1_8',
       'targtype1_9', 'targtype1_10', 'targtype1_11', 'targtype1_12',
       'targtype1_13', 'targtype1_14', 'targtype1_15', 'targtype1_16',
       'targtype1_17', 'targtype1_18', 'targtype1_19', 'targtype1_20',
       'targtype1_21', 'targtype1_22', 'country_4',
'country_5',
'country_6',
'country_14',
'country_15',
'country_16',
'country_18',
'country_19',
'country_21',
'country_28',
'country_32',
'country_33',
'country_37',
'country_38',
'country_41',
'country_42',
'country_43',
'country_45',
'country_55',
'country_56',
'country_60',
'country_65',
'country_68',
'country_69',
'country_74',
'country_75',
'country_78',
'country_92',
'country_93',
'country_94',
'country_95',
'country_97',
'country_98',
'country_102',
'country_104',
'country_106',
'country_107',
'country_110',
'country_113',
'country_121',
'country_122',
'country_123',
'country_130',
'country_137',
'country_138',
'country_141',
'country_142',
'country_146',
'country_147',
'country_151',
'country_153',
'country_155',
'country_156',
'country_160',
'country_161',
'country_167',
'country_173',
'country_182',
'country_183',
'country_185',
'country_186',
'country_195',
'country_198',
'country_200',
'country_201',
'country_202',
'country_203',
'country_205',
'country_208',
'country_209',
'country_210',
'country_213',
'country_214',
'country_217',
'country_228',
'country_229',
'country_603',
'country_1004', 'nkill']]
#42
y = datasettt[['gname_Abu Sayyaf Group (ASG)',
'gname_Al-Nusrah Front',
'gname_Al-Qaida in the Arabian Peninsula (AQAP)',
'gname_Al-Shabaab',
'gname_Allied Democratic Forces (ADF)',
'gname_Anarchists',
'gname_Anti-Balaka Militia',
'gname_Anti-Muslim extremists',
'gname_Baloch Liberation Army (BLA)',
'gname_Baloch Liberation Front (BLF)',
'gname_Baloch Republican Army (BRA)',
'gname_Bangsamoro Islamic Freedom Movement (BIFM)',
'gname_Barisan Revolusi Nasional (BRN)',
'gname_Barqa Province of the Islamic State',
'gname_Boko Haram',
'gname_Communist Party of India - Maoist (CPI-Maoist)',
'gname_Communist Party of Nepal - Maoist (CPN-Maoist-Chand)',
"gname_Donetsk People's Republic",
'gname_Free Syrian Army',
'gname_Fulani extremists',
'gname_Garo National Liberation Army',
'gname_Gorkha Janmukti Morcha (GJM)',
'gname_Hamas (Islamic Resistance Movement)',
'gname_Hizbul Mujahideen (HM)',
'gname_Houthi extremists (Ansar Allah)',
'gname_Islamic State of Iraq and the Levant (ISIL)',
'gname_Janjaweed',
'gname_Jihadi-inspired extremists',
'gname_Khorasan Chapter of the Islamic State',
"gname_Kurdistan Workers' Party (PKK)",
'gname_Lashkar-e-Taiba (LeT)',
"gname_Lord's Resistance Army (LRA)",
"gname_Luhansk People's Republic",
'gname_Maoists',
'gname_Mozambique National Resistance Movement (MNR)',
'gname_Muslim extremists',
'gname_National Liberation Army of Colombia (ELN)',
"gname_New People's Army (NPA)",
'gname_Palestinian Extremists',
'gname_Pro Hartal Activists',
'gname_Revolutionary Armed Forces of Colombia (FARC)',
'gname_Runda Kumpulan Kecil (RKK)',
'gname_Separatists',
'gname_Sinai Province of the Islamic State',
"gname_Sudan People's Liberation Movement in Opposition (SPLM-IO)",
'gname_Taliban',
'gname_Tehrik-i-Taliban Pakistan (TTP)',
'gname_Tribesmen',
'gname_Tripoli Province of the Islamic State',
'gname_United Liberation Front of Assam (ULFA)']]

#.values.reshape(-1, 1)

#10
print(X.shape, y.shape) 

(24070, 120) (24070, 50)


In [158]:
len(y.columns)

50

In [159]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

X_train.head()

,attacktype1_1,attacktype1_2,attacktype1_3,attacktype1_4,attacktype1_5,attacktype1_6,attacktype1_7,attacktype1_8,attacktype1_9,weaptype1_2,...,country_209,country_210,country_213,country_214,country_217,country_228,country_229,country_603,country_1004,nkill
49590,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,31.0
24943,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,26.0
24307,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,4.0
33722,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
53996,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4.0


In [160]:
# from sklearn.preprocessing import StandardScaler
# X_scaler = StandardScaler().fit(X_train)
# y_scaler = StandardScaler().fit(y_train)

In [161]:
# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)
# y_train_scaled = y_scaler.transform(y_train)
# y_test_scaled = y_scaler.transform(y_test)

In [162]:
# from sklearn.linear_model import LinearRegression
# model = LinearRegression()
# model.fit(X_train, y_train)
# plt.scatter(model.predict(X_train), model.predict(X_train) - y_train, c="blue", label="Training Data")
# plt.scatter(model.predict(X_test), model.predict(X_test) - y_test, c="orange", label="Testing Data")
# plt.legend()
# plt.hlines(y=0, xmin=y_test.min(), xmax=y_test.max())
# plt.title("Residual Plot")
# plt.show()

In [163]:
# from sklearn.metrics import mean_squared_error

# predictions = model.predict(X_test)
# MSE = mean_squared_error(y_test, predictions)
# r2 = model.score(X_test, y_test)

# print(f"MSE: {MSE}, R2: {r2}")

In [172]:
from tensorflow.keras.models import Sequential

model = Sequential()

In [173]:
from tensorflow.keras.layers import Dense
number_inputs = 120
number_hidden_nodes = 1000
model.add(Dense(units=number_hidden_nodes,
                activation='relu', input_dim=number_inputs))
model.add(Dense(units=number_hidden_nodes,
                activation='relu'))

In [174]:
number_classes = 50
model.add(Dense(units=number_classes, activation='softmax'))

In [175]:
model.summary() 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 1000)              121000    
_________________________________________________________________
dense_25 (Dense)             (None, 1000)              1001000   
_________________________________________________________________
dense_26 (Dense)             (None, 50)                50050     
Total params: 1,172,050
Trainable params: 1,172,050
Non-trainable params: 0
_________________________________________________________________


In [176]:
model.compile(optimizer='adam',
              loss='mse',
              metrics=['accuracy'])

In [177]:
from tensorflow.keras.utils import to_categorical 
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)
y_train_categorical

array([[[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [0., 1.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]],

       ...,

       [[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]], dtype=float32)

In [178]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

model.fit(
    X_train,
    y_train,
    epochs=10,
    shuffle=True,
    verbose=2
)

Epoch 1/10
 - 25s - loss: 0.0077 - acc: 0.7252
Epoch 2/10
 - 24s - loss: 0.0055 - acc: 0.8146
Epoch 3/10
 - 24s - loss: 0.0050 - acc: 0.8295
Epoch 4/10
 - 24s - loss: 0.0050 - acc: 0.8297
Epoch 5/10
 - 24s - loss: 0.0048 - acc: 0.8354
Epoch 6/10
 - 24s - loss: 0.0046 - acc: 0.8395
Epoch 7/10
 - 24s - loss: 0.0046 - acc: 0.8436
Epoch 8/10
 - 27s - loss: 0.0045 - acc: 0.8460
Epoch 9/10
 - 31s - loss: 0.0044 - acc: 0.8484
Epoch 10/10
 - 26s - loss: 0.0044 - acc: 0.8502


In [93]:
y_pred=model.predict_classes(X_test[19316])
print(y_pred)
baddies={0:"Qaida",1:"Al=Shabaab",2:"Boko Haram",3:"Donetsk People's Republic",4:"Houthi extremists (Ansar Allah)",5:"Islamic State of Iraq and the Levant (ISIL)",6:"Kurdistan Workers' Party (PKK)",7:"Maoists",8:"New People's Army (NPA)",9:"Taliban"}



KeyError: 19316

In [92]:
y_test

,attacktype1_1,attacktype1_2,attacktype1_3,attacktype1_4,attacktype1_5,attacktype1_6,attacktype1_7,attacktype1_8,attacktype1_9,weaptype1_2,...,targtype1_13,targtype1_14,targtype1_15,targtype1_16,targtype1_17,targtype1_18,targtype1_19,targtype1_20,targtype1_21,targtype1_22
19316,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
46255,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
12354,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
41454,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
21171,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4114,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22094,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44708,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16964,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
23558,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


array(['Unknown', 'Unknown', 'Unknown', ..., 'Unknown', 'Unknown',
       'Unknown'], dtype=object)